In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time
import requests
import requests
import json
import pandas as pd

pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)  # Sin límite de columnas visibles
pd.set_option('display.width', 0)  # Configurar ancho dinámico para la pantalla
pd.set_option('display.max_rows', 100)

import os, sys
sys.path.append(r"C:\Users\jnavarro\Solaria Energía y Medio Ambiente\00-GEN - Documentos\Base de Datos\python")
from utils.connector import execute_query, insertar_dataframe_en_mysql

### Funciones de extraccion y limpieza

In [36]:
def get_df_indicadores_raw(start_date, end_date, indicator_ids):
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')
    query_indicadores = ", ".join([f"'{x}'" for x in indicator_ids])

    query = f"""SELECT Datetime_utc, indicator_id, geo_id, magnitud_id, value FROM t_api_esios_indicadores_data 
                WHERE Datetime BETWEEN '{start_date_str}' AND '{end_date_str}'
                AND indicator_id IN ({query_indicadores})"""
    df = execute_query(query, 'esios')

    return df

In [37]:
def limpieza_df(df):
    global indicator_id_dict
    df_clean = df.copy()

    df_clean['Datetime_utc'] = pd.to_datetime(df_clean['Datetime_utc'])
    df_clean['Datetime_hour'] = df_clean['Datetime_utc'].dt.strftime('%Y-%m-%d %H:00:00')

    df_precios_potencia = df_clean.loc[df_clean['magnitud_id']!=13]
    df_energia = df_clean.loc[df_clean['magnitud_id']==13]

    # Agrupar por horas
    df_energia_hour = df_energia.groupby(['Datetime_hour', 'indicator_id'], as_index=False)['value'].sum()
    df_precios_potencia_hour = df_precios_potencia.groupby(['Datetime_hour', 'indicator_id'], as_index=False)['value'].mean()

    df_clean_hour = pd.concat([df_energia_hour, df_precios_potencia_hour], axis=0)

    # Hacer pivot - 1 columna por indicador
    df_clean_hour_pivot = df_clean_hour.pivot(index='Datetime_hour', columns='indicator_id', values='value').reset_index()
    

    # Renombrar columnas
    df_clean_hour_pivot.rename(columns=indicator_id_dict, inplace=True)

    cols_round = [col for col in df_clean_hour_pivot.columns if col not in ['Datetime_hour', 'MD', 'IDA1', 'IDA2']]
    df_clean_hour_pivot[cols_round] = df_clean_hour_pivot[cols_round].round(1)
    return df_clean_hour_pivot

In [38]:
start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 8, 1)
indicator_id_dict = {
    10257: 'Gen.P48 Total',
    10010: 'Gen.P48 Eolica',
    84: 'Gen.P48 Fotovoltaica',
    10027: 'Demanda P48',
    10026: 'Interconexiones P48',
    612: 'IDA1',
    613: 'IDA2',
    600: 'MD'
}

indicator_ids = list(indicator_id_dict.keys())
df = get_df_indicadores_raw(start_date, end_date, indicator_ids)

df_final = limpieza_df(df)

In [39]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13872 entries, 0 to 13871
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Datetime_hour         13872 non-null  object 
 1   Gen.P48 Fotovoltaica  13108 non-null  float64
 2   MD                    13872 non-null  float64
 3   IDA1                  13464 non-null  float64
 4   IDA2                  13656 non-null  float64
 5   Gen.P48 Eolica        13836 non-null  float64
 6   Interconexiones P48   13872 non-null  float64
 7   Demanda P48           13855 non-null  float64
 8   Gen.P48 Total         13836 non-null  float64
dtypes: float64(8), object(1)
memory usage: 975.5+ KB


### Tratamiento de nulos

Generacion Fotovoltaica

In [40]:
df_final.loc[df_final['Gen.P48 Fotovoltaica'].isnull()]

indicator_id,Datetime_hour,Gen.P48 Fotovoltaica,MD,IDA1,IDA2,Gen.P48 Eolica,Interconexiones P48,Demanda P48,Gen.P48 Total
3623,2024-05-30 22:00:00,NaN,10.00,9.00,5.90,13250.3,-3360.2,25249.2,28569.9
3624,2024-05-30 23:00:00,NaN,5.50,4.50,5.01,12854.8,-3454.0,23949.2,27422.2
3625,2024-05-31 00:00:00,NaN,4.77,4.62,5.00,12757.7,-3650.0,23237.7,26954.2
3626,2024-05-31 01:00:00,NaN,3.25,0.67,3.25,12370.9,-2954.1,22688.3,25776.4
3627,2024-05-31 02:00:00,NaN,3.25,0.02,3.25,11689.2,-2470.5,22475.9,24808.4
...,...,...,...,...,...,...,...,...,...
13248,2025-07-05 23:00:00,NaN,106.01,100.54,97.27,7158.8,-2114.2,25925.5,28468.7
13249,2025-07-06 00:00:00,NaN,103.60,92.38,91.68,6716.9,-1580.0,24484.9,26681.4
13251,2025-07-06 02:00:00,NaN,100.10,90.98,85.76,5519.6,-1225.6,22753.9,24745.9
13252,2025-07-06 03:00:00,NaN,100.82,92.57,92.77,5082.6,-1597.9,22565.6,24567.5


In [41]:
# Los valores nulos corresponden a horas no solares, así que reemplazamos por 0
df_final['Gen.P48 Fotovoltaica'] = df_final['Gen.P48 Fotovoltaica'].fillna(0.0)

Generacion Total

In [42]:
df_final.loc[df_final['Gen.P48 Total'].isnull()]

indicator_id,Datetime_hour,Gen.P48 Fotovoltaica,MD,IDA1,IDA2,Gen.P48 Eolica,Interconexiones P48,Demanda P48,Gen.P48 Total
11603,2025-04-28 10:00:00,0.0,-1.01,-0.01,0.00,NaN,-1943.4,-2118.4,NaN
11604,2025-04-28 11:00:00,0.0,-2.50,-2.50,-1.00,NaN,236.0,236.0,NaN
11605,2025-04-28 12:00:00,0.0,-3.00,-2.27,-0.01,NaN,802.2,277.2,NaN
11606,2025-04-28 13:00:00,0.0,-1.54,0.00,0.00,NaN,1418.7,1418.7,NaN
11607,2025-04-28 14:00:00,0.0,-0.30,-5.77,-0.01,NaN,1433.5,1433.5,NaN
11608,2025-04-28 15:00:00,0.0,-0.01,-1.01,0.00,NaN,1174.2,1174.2,NaN
11609,2025-04-28 16:00:00,0.0,0.01,0.65,1.72,NaN,1837.8,1837.8,NaN
11610,2025-04-28 17:00:00,0.0,5.20,15.20,15.46,NaN,1515.8,1515.8,NaN
11611,2025-04-28 18:00:00,0.0,40.70,47.37,58.23,NaN,950.5,950.5,NaN
11612,2025-04-28 19:00:00,0.0,80.80,80.79,85.80,NaN,853.2,853.2,NaN


In [43]:
# Corresponde al día del apagón. Lo cual también explica los nulos en Gen. Eolica y Demanda
# Es una situación excepcional pero sustituimos también por 0 para replicar la realidad
df_final[['Gen.P48 Total', 'Gen.P48 Eolica', 'Demanda P48']] = df_final[['Gen.P48 Total', 'Gen.P48 Eolica', 'Demanda P48']].fillna(0.0)

Precios IDA1 y IDA2

In [45]:
df_final.loc[(df_final['IDA1'].isnull()) | (df_final['IDA2'].isnull())]

indicator_id,Datetime_hour,Gen.P48 Fotovoltaica,MD,IDA1,IDA2,Gen.P48 Eolica,Interconexiones P48,Demanda P48,Gen.P48 Total
3959,2024-06-13 22:00:00,0.0,78.86,NaN,88.86,6255.6,-338.4,23897.4,24814.8
3960,2024-06-13 23:00:00,0.0,76.44,NaN,80.75,5998.5,-331.7,22826.6,23537.4
3961,2024-06-14 00:00:00,0.0,75.72,NaN,75.72,5748.5,-363.9,22117.9,22734.8
3962,2024-06-14 01:00:00,0.0,75.30,NaN,75.30,5527.8,-380.1,21719.6,22272.2
3963,2024-06-14 02:00:00,0.9,75.72,NaN,75.72,5223.0,-351.4,21591.8,22096.2
...,...,...,...,...,...,...,...,...,...
13722,2025-07-25 17:00:00,12742.8,30.85,NaN,79.99,9239.3,-5845.2,28682.7,37187.0
13723,2025-07-25 18:00:00,5045.6,85.00,NaN,93.50,10693.0,-5101.3,29173.8,34889.6
13724,2025-07-25 19:00:00,475.8,104.72,NaN,99.72,12192.2,-3776.1,29227.4,34284.4
13725,2025-07-25 20:00:00,0.2,101.94,NaN,98.55,13264.8,-3470.3,28297.4,33423.7


In [46]:
# Sustituimos los valores nulos del IDA1 por los del IDA2 y viceversa
# Cuando ambos son nulos, tomamos el valor del MD
df_final['IDA1'] = np.where(
    df_final['IDA1'].isnull(),
    np.where(df_final['IDA2'].isnull(), df_final['MD'], df_final['IDA2']),
    df_final['IDA1']
)
df_final['IDA2'] = np.where(
    df_final['IDA2'].isnull(),
    np.where(df_final['IDA1'].isnull(), df_final['MD'], df_final['IDA1']),
    df_final['IDA2']
)

In [47]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13872 entries, 0 to 13871
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Datetime_hour         13872 non-null  object 
 1   Gen.P48 Fotovoltaica  13872 non-null  float64
 2   MD                    13872 non-null  float64
 3   IDA1                  13872 non-null  float64
 4   IDA2                  13872 non-null  float64
 5   Gen.P48 Eolica        13872 non-null  float64
 6   Interconexiones P48   13872 non-null  float64
 7   Demanda P48           13872 non-null  float64
 8   Gen.P48 Total         13872 non-null  float64
dtypes: float64(8), object(1)
memory usage: 975.5+ KB


### Guardar los datos en un archivo CSV

In [50]:
df_final.to_csv('data_training/esios_data_modelo_dl.csv', index=False)